# RAG Pipeline: Evaluation and Analysis

This notebook focuses on running the complete Retrieval-Augmented Generation (RAG) pipeline on a set of representative questions. We will analyze how the system retrieves relevant complaints and generates answers using the LLM.

### Pipeline Components:
1. **Vector Store**: FAISS index containing complaint embeddings.
2. **Retriever**: Similarity search mechanism to find top-K relevant documents.
3. **LLM**: Instruction-tuned model (Phi-3 Mini) to generate answers based on context.
4. **Prompt Engineering**: Structured templates to guide the LLM's response.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import IPython
from IPython.display import display, Markdown

# allow imports from project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src import config, vectorstore, llm
from src.rag_pipline import RAGPipeline

config.setup_hf_cache()
print("✓ Environment setup complete")

[OK] HuggingFace cache set to: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf
✓ Environment setup complete


## 1. Initialize Production Pipeline

We initialize the `RAGPipeline` class, which encapsulates the loading of the vector store and the LLM engine.

In [2]:
pipeline = RAGPipeline()
if pipeline.initialize():
    print("✓ RAG Pipeline initialized and ready!")
else:
    print("❌ Pipeline initialization failed. Check your vector store and model paths.")


--- Initializing RAG Pipeline ---
Attempting to load vector store from: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
  (First run will download ~80MB to c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf)
[OK] Embedding model loaded
Loading existing FAISS index from c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss...
[OK] Vector store loaded (ntotal=1,375,327)
Loading Alternative LLM: microsoft/Phi-3-mini-4k-instruct
  Max new tokens: 512
  Temperature: 0.1


tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for thi

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

## 2. Representative Evaluation Questions

Here is a curated list of questions that represent common consumer concerns. We will use these to evaluate the system's performance.

In [3]:
eval_questions = [
    "What are common issues consumers face with credit card interest rates?",
    "How do customers describe problems with mortgage loan modifications?",
    "What are the most frequent complaints regarding unauthorized transactions on checking accounts?",
    "What reasons do consumers give for disputes regarding credit reporting inaccuracies?",
    "What are the common themes in complaints against debt collection agencies' practices?",
    "How do consumers react to unexpected fees on prepaid cards?",
    "What are the typical complaints about vehicle loan servicing?",
    "What issues do consumers report when trying to close their savings accounts?"
]

print(f"✓ Defined {len(eval_questions)} representative questions.")

✓ Defined 8 representative questions.


## 3. Running RAG Pipeline and Results Analysis

For each question, we run the full RAG cycle:
1. **Retrieve**: Find top 5 relevant complaint chunks.
2. **Augment**: Format the chunks into a context string for the prompt.
3. **Generate**: Use Phi-3 Mini to generate a professional answer based *only* on the context.

In [4]:
all_results = []

for i, query in enumerate(eval_questions, 1):
    display(Markdown(f"### Question {i}: {query}"))
    
    # Run the pipeline
    result = pipeline.run(query, k=5)
    
    # Display the answer
    display(Markdown(f"**Generated Answer:** {result['answer']}"))
    
    # Display source metadata summary
    sources = result['source_documents']
    display(Markdown(f"*Retrieved {len(sources)} sources from products: {set([doc.metadata.get('product', 'N/A') for doc in sources])}*"))
    
    # Show a snippet of the most relevant source
    top_source = sources[0]
    display(Markdown(f"**Top Source Snippet (Complaint {top_source.metadata.get('complaint_id')}):**  \n*{top_source.page_content[:250]}...*"))
    
    display(Markdown("---"))
    
    all_results.append({
        "question": query,
        "answer": result['answer'],
        "sources": sources
    })

### Question 1: What are common issues consumers face with credit card interest rates?

Processing Query: 'What are common issues consumers face with credit card interest rates?'
Generating answer based on 5 sources...


**Generated Answer:** a credit score of less than . the interest rate was . however, i have had the card for more than two years, i have had a credit score of less than .

*Retrieved 5 sources from products: {'Credit card', 'Credit card or prepaid card'}*

**Top Source Snippet (Complaint 3580921):**  
*but not going through where the big issues are ... the interest rates that these greedy credit card companies retail store cards are charging. these are the ones to be looked upon with more scrutiny. not the math models that you develop and ask banks...*

...

Processing Query: 'What issues do consumers report when trying to close their savings accounts?'
Generating answer based on 5 sources...


**Generated Answer:** numerous banking errors on atm machines, fees, confidentiality and customer service issues.

*Retrieved 5 sources from products: {'Money transfer, virtual currency, or money service', 'Checking or savings account'}*

**Top Source Snippet (Complaint 13049257):**  
*care that i closed the saving account on the spot. the only thing that i was not surprised of is when the customer support showed no indifference in her attitude as i asked to close the account. one would think that a company would try to retain a cu...*

---

## 4. Pipeline Evaluation Table

We consolidate the results into a structured evaluation table. This can be exported to your report. 

> **Note:** The `Quality Score` and `Comments/Analysis` columns are intended for manual review to ensure the AI's answers are grounded and accurate.

In [5]:
evaluation_data = []

for res in all_results:
    # Get 1-2 source IDs
    source_ids = [doc.metadata.get('complaint_id', 'N/A') for doc in res['sources'][:2]]
    
    evaluation_data.append({
        "Question": res['question'],
        "Generated Answer": res['answer'],
        "Retrieved Sources": ", ".join(map(str, source_ids)),
        "Quality Score (1-5)": "",  # Placeholder for manual review
        "Comments/Analysis": ""      # Placeholder for manual review
    })

df_eval = pd.DataFrame(evaluation_data)

# Set column display for better readability in the notebook
pd.set_option('display.max_colwidth', 300)
display(df_eval)

# Optional: Save to CSV for the report
# df_eval.to_csv("rag_evaluation_report.csv", index=False)

,Question,Generated Answer,Retrieved Sources,Quality Score (1-5),Comments/Analysis
0,What are common issues consumers face with credit card interest rates?,"a credit score of less than . the interest rate was . however, i have had the card for more than two years, i have had a credit score of less than .","3580921, 3265164",,
1,How do customers describe problems with mortgage loan modifications?,"Wells Fargo refused to offer me any mortgage modifications, to negotiate, or offer any assistance for that matter. I qualified for modifications, and had funds to make good on the loan payments. instead, the house i owned for 22 years, was sold.","2758234, 1879110",,
2,What are the most frequent complaints regarding unauthorized transactions on checking accounts?,Is there anything else I can do about the fraud alerts and communication?,"11439123, 11725491",,
3,What reasons do consumers give for disputes regarding credit reporting inaccuracies?,"Document 5 --- credit reporting agencies, including and experian. recent lawsuit findings in separate rulings, courts have determined that and experian failed to maintain reasonable procedures to ensure maximum accuracy of consumer credit information. these findings highlight systemic issues wit...","8107540, 11827412",,
4,What are the common themes in complaints against debt collection agencies' practices?,"the collection company filed a complaint with the court. i attempted to contact the collection companies attorneys of record on several occasions including , 2015 and 2015 to discuss first and foremost the violations of various creditor and debtor laws including but not limited to the fair debt ...","10321884, 10949444",,
5,How do consumers react to unexpected fees on prepaid cards?,"hrough the purchase and non-usage of prepaid cards as sold in america 's top retailers. this is a gross misuse of public and consumer trust, and the only purpose of this complaint to is to put an end to the corporate abuse of consumer 's hard earned money.","8305869, 2683836",,
6,What are the typical complaints about vehicle loan servicing?,e of the complaint,"10285038, 2374626",,
7,What issues do consumers report when trying to close their savings accounts?,"numerous banking errors on atm machines, fees, confidentiality and customer service issues.","13049257, 7136583",,
